In [9]:
import io, os
import re as re
import zipfile as zipfile
import string
from collections import defaultdict, Counter
mytextzip = ''
corpus = []
labels = []
def readfromzip(zipname: str, n_docs: int) -> None:
    global mytextzip
    global c
    with zipfile.ZipFile(zipname) as z:
        for zipinfo in z.infolist():
            if zipinfo.filename.endswith('.txt') and re.search('raw_texts', zipinfo.filename):
                with z.open(zipinfo) as f:
                    textfile = io.TextIOWrapper(f, encoding='cp1254', newline='\r\n')
                    for line in textfile:
                        if len(line):
                            if re.search(r'([a-zA-Z]+\r\n)', line):
                                mytextzip += line.strip() + ' '
                                continue
                            mytextzip += line.strip()
                    corpus.append(mytextzip)
                    labels.append(zipinfo.filename.split("/")[2])
                    if len(corpus) >= n_docs:
                        break
                    mytextzip =''
readfromzip('film_yorumlari.zip',1500)

In [10]:
for t in range(len(corpus)):
    corpus[t] = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', corpus[t].lower())
    for i in range(len(corpus[t])):
        corpus[t][i] = "<SOS> " + corpus[t][i] + " <EOS>"
    corpus[t] = ' '.join(corpus[t])

In [11]:
import numpy as np
def split(X, y, test_size, set_seed):
    arrays = [X,y]
    assert all(len(arr) == len(arrays[0]) for arr in arrays)
    seed = np.random.randint(0, 2**(32 - 1) - 1) if set_seed < 0 else set_seed
    
    for arr in arrays:
        rstate = np.random.RandomState(seed)
        rstate.shuffle(arr)
        
    size = int(len(X)*test_size)
    x_train, x_test = X[size:], X[:size]
    y_train, y_test = y[size:], y[:size]
    return x_train, x_test, y_train, y_test

In [12]:
X_train, X_test, y_train, y_test = split(corpus, labels, 0.2, -1)

In [14]:
# train_runtime_complexity = O(len(corpus)*(len(text)-4)*n)
import math, sys
class Ngrams():
    def __init__(self, n):
        self.n = n
        
    
    def _train(self, corpus, labels):
        awd = defaultdict(lambda: defaultdict(int))
        ngrams = defaultdict(lambda: defaultdict(list))
        self.labels = labels
        for i, text in enumerate(corpus):
            label = self.labels[i]
            text = text.split()
            for i in range(len(text)-self.n+1):
                before = ' '.join([text[i+j] for j in range(self.n-1)])
                awd[label][before] += 1
                ngrams[label][before].append(text[i+self.n-1])
            pre = ' '.join([text[-i] for i in range(1, self.n)])
            awd[label][pre] += 1
        for a in ngrams.keys():
            for k in ngrams[a].keys():
                ngrams[a][k] = Counter(ngrams[a][k])
                ngrams[a][k] = (awd[a][k], ngrams[a][k])
        return ngrams
    
    def train(self, corpus, labels):
        self.ngrams = self._train(corpus, labels)
                
    def _predict(self, text):
        prob = -sys.maxsize
        res = None
        text = text.split()
        for label in set(self.labels):
            newprob = 0
            for i in range(len(text)-1):
                try:
                    before = ' '.join([text[i+j] for j in range(self.n-1)])
                    totalC = self.ngrams[label][before][0]
                    togetherC = self.ngrams[label][before][1][text[i+self.n-1]]
                    ans = togetherC / totalC
                    newprob += math.log(ans)
                except:
                    newprob += math.log(1e-10)
            if newprob > prob:
                prob = newprob
                res = label
        return res
        
    def predict(self, corpus):
        self.preds = []
        for t in range(len(corpus)):
            self.preds.append(self._predict(corpus[t]))
        return self.preds

In [16]:
ng = Ngrams(2)
ng.train(X_train, y_train)


In [17]:
pred = ng.predict(X_test)

In [19]:
def accuracy(y_true, y_pred):
    acc = 0
    assert len(y_true) == len(y_pred)
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            acc += 1
    print(str(acc) + "/" + str(len(y_test)))
    return (acc*100)/len(y_true)

In [20]:
accuracy(y_test, pred)

10/21


47.61904761904762